## Apprentissage supervisé: Forêts d'arbres aléatoires (Random Forests)

Intéressons nous maintenant à un des algorithmes les plus popualires de l'état de l'art. Cet algorithme est non-paramétrique et porte le nom de **forêts d'arbres aléatoires**

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

plt.style.use('seaborn')

## A l'origine des forêts d'arbres aléatoires : l'arbre de décision

Les fôrets aléatoires appartiennent à la famille des méthodes d'**apprentissage ensembliste** et sont construits à partir d'**arbres de décision**. Pour cette raison, nous allons tout d'abord présenter les arbres de décisions.

Un arbre de décision est une manière très intuitive de résoudre un problème de classification. On se contente de définir un certain nombre de questions qui vont permetre d'identifier la classe adéquate.

In [ ]:
import fig_code.figures as fig
fig.plot_example_decision_tree()

Le découpage binaire des données est rapide a mettre en oeuvre. La difficulté va résider dans la manière de déterminer quelle est la "bonne" question à poser.

C'est tout l'enjeu de la phase d'apprentissage d'un arbre de décision. L'algorithme va déterminer, au vue d'un ensemble de données, quelle question (ou découpage...) va apporter le plus gros gain d'information.

### Construction d'un arbre de décision
Voici un exemple de classifieur à partir d'un arbre de décision en utlisiant la libraire scikit-learn.

Nous commencons par définir un jeu de données en 2 dimensions avec des labels associés:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

Nous avons précemment défini une fonction qui va faciliter la visualisation du processus :

In [ ]:
from fig_code.figures import visualize_tree, plot_tree_interactive

On utilise maintenant le module ``interact`` dans Ipython pour visualiser les découpages effectués par l'arbre de décision en fonction de la profondeur de l'arbre (*depth* en anglais), i.e. le nombre de questions que l'arbre peut poser :

In [ ]:
plot_tree_interactive(X, y);

**Remarque** : à chaque augmentation de la profondeur de l'arbre, chaque branche est découpée en deux **à l'expection** des branches qui contiennent uniquement des points d'une unique classe.

L'arbre de décision est une méthode de classification non paramétrique facile à mettre en oeuvre

**Question: Observez-vous des problèmes avec cette modélisation ?**

## Arbre de décision et sur-apprentissage

Un problème avec les arbres de décision est qu'ils ont tendance à **sur-apprendre** rapidement sur les données d'apprentissage. En effet, ils ont une forte tendance à capturer le bruit présent dans les données plutôt que la vraie distribution recherchée. Par exemple, si on construit 2 arbres à partir de sous ensembles des données définies précédemment, on obtient les deux classifieurs suivants:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

plt.figure()
visualize_tree(clf, X[:200], y[:200], boundaries=False)
plt.figure()
visualize_tree(clf, X[-200:], y[-200:], boundaries=False)

Les 2 classifieurs ont des différences notables si on regarde en détails les figures. Lorsque'on va prédire la classe d'un nouveau point, cela risque d'être impacté par le bruit dans les données plus que par le signal que l'on cherche à modéliser.


## Prédictions ensemblistes: Forêts aléatoires
Une façon de limiter ce problème de sur-apprentissage est d'utiliser un **modèle ensembliste**: un méta-estimateur qui va aggréger les predictions de mutliples estimateurs (qui peuvent sur-apprendre individuellement). Grace à des propriétés mathématiques plutôt magiques (!), la prédiction aggrégée de ces estimateurs s'avère plus performante et robuste que les performances des estimateurs considérés individuellement.

Une des méthodes ensemblistes les plus célèbres est la méthode des **forêts d'arbres aléatoires** qui aggrège les prédictions de multiples arbres de décision.

Il y a beaucoup de littératures scientifiques pour déterminer la façon de rendre aléatoires ces arbres mais donner un exemple concret, voici un ensemble de modèle qui utilise seulement un sous échantillon des données :


In [ ]:
X, y = make_blobs(n_samples=300, centers=4,
                      random_state=0, cluster_std=2.0)

def fit_randomized_tree(random_state=0):
    rng = np.random.RandomState(random_state)
    i = np.arange(len(y))
    rng.shuffle(i)
    
    clf = DecisionTreeClassifier(max_depth=5)
    #on utilise seulement 250 exemples choisis aléatoirement sur les 300 disponibles
    visualize_tree(clf, X[i[:250]], y[i[:250]], boundaries=False,
                   xlim=(X[:, 0].min(), X[:, 0].max()),
                   ylim=(X[:, 1].min(), X[:, 1].max()))
    
from ipywidgets import interact
interact(fit_randomized_tree, random_state=(0, 100));

On peut observer dans le détail les changements du modèle en fonction du tirage aléatoire des données qu'il utilise pour l'apprentissage, alors que la distribution des données est figée !

La forêt aléatoire va faire des caluls similaires, mais va aggréger l'ensemble des arbres aléatoires générés pour construire une unique prédiction:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
visualize_tree(clf, X, y, boundaries=False);

In [ ]:
from sklearn.svm import SVC  
clf = SVC(kernel='linear')
clf.fit(X, y)
visualize_tree(clf,X, y, boundaries=False)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=200, facecolors='none');

En moyennant 100 arbres de décision "perturbés" aléatoirement, nous obtenons une prédiction aggrégé qui modélise avec plus de précision nos données.

*(Remarque: ci dessus, notre perturbation aléatoire est effectué en echantillonant de manière aléatoire nos données... Les arbres aléatoires utilisent des techniques plus sophistiquées, pour plus de détails voir la [documentation de scikit-learn](http://scikit-learn.org/stable/modules/ensemble.html#forest)*)

## Exemple 1 : utilisation en régression
On considère pour cet exemple un cas d'tétude différent des exemples précédent de classification. Les arbres aléatoires peuvent être également utilisés sur des problèmes de régression (c'est à dire la prédiction d'une variable continue plutôt que discrète).

L'estimateur que nous utiliserons est le suivant: ``sklearn.ensemble.RandomForestRegressor``.

Nous présentons rapidement comment il peut être utilisé:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# On commence par créer un jeu de données d'apprentissage
x = 10 * np.random.rand(100)

def model(x, sigma=0.):
    # sigma controle le bruit 
    # sigma=0 pour avoir une distribution "parfaite"
    oscillation_rapide = np.sin(5 * x)
    oscillation_lente = np.sin(0.5 * x)
    bruit = sigma * np.random.randn(len(x))

    return oscillation_rapide + oscillation_lente + bruit

y = model(x)
plt.figure(figsize=(10,5))
plt.scatter(x, y);

In [ ]:
xfit = np.linspace(0, 10, num=1000)
# yfit contient les prédictions de la forêt aléatoire à partir des données bruités
yfit = RandomForestRegressor(100).fit(x[:, None], y).predict(xfit[:, None])
# ytrue contient les valuers du modèle qui génèrent nos données avec un bruit nul 
ytrue = model(xfit, sigma=0) 

plt.figure(figsize=(10,5))
#plt.scatter(x, y)
plt.plot(xfit, yfit, '-r', label = 'forêt aléatoire')
plt.plot(xfit, ytrue, '-g', alpha=0.5, label = 'distribution non bruitée')
plt.legend();

On observe que les forêts aléatoires, de manière non-paramétrique, arrivent à estimer une distribution avec de mutliples périodicités sans aucune intervention de notre part pour définir ces périodicités !

---
**Hyperparamètres**

Utilisons l'outil d'aide inclus dans Ipython pour explorer la classe ``RandomForestRegressor``. Pour cela on rajoute un ? à la fin de l'objet

In [ ]:
RandomForestRegressor?

Quelle sont les options disponibles  pour le ``RandomForestRegressor``?
Quelle influence sur le graphique précédent si on modifie ces valeurs?

Ces paramètres de classe sont appelés les **hyperparamètres** d'un modèle.

---

In [ ]:
# Exercice : proposer un modèle de régression à vecteur support permettant de modéliser le phénomène
from sklearn.svm import SVR

SVMreg = SVR().fit(x[:, None], y)

yfit_SVM = SVMreg.predict(xfit[:, None])

plt.figure(figsize=(10,5))
plt.scatter(x, y)
plt.plot(xfit, yfit_SVM, '-r', label = 'SVM')
plt.plot(xfit, ytrue, '-g', alpha=0.5, label = 'distribution non bruitée')
plt.legend();

In [ ]:
SVR?